In [1]:
import pandas as pd
from tqdm.autonotebook import tqdm

/tmp/ipykernel_246992/158874326.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
from openai import OpenAI
client = OpenAI(api_key="API_KEY")

In [3]:
aspects = ["food", "service", "ambience", "price", "other"]
sentiments = ["positive", "neutral", "negative"]

In [4]:
df = pd.read_csv("../../eval/results/phi3_n_shot/phi3_two_shot_df.csv")
fl = []
for i, x in df.iterrows():
    fl.append({k:v for k,v in eval(x["true_labels"]).items() if v is not None})
df["true_labels"] = fl

In [5]:
df = df[["text", "true_labels"]]

In [6]:
df.head()

,text,true_labels
0,bread top notch well,{'food': 'positive'}
1,say one fastest delivery times city,{'service': 'positive'}
2,food always fresh ready eat,{'food': 'positive'}
3,mention coffee outstanding,{'food': 'positive'}
4,trust people go sushi never disappoints,{'other': 'positive'}


In [7]:
df.shape

(749, 2)

In [ ]:
openai_response = []

for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    label = row["true_labels"]
    text = row["text"]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125", 
        response_format={ "type": "json_object" }, 
        messages=[
            {"role": "system", "content": f"Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. Output should only contain mentioned aspects and their respective sentiments as a json. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation."},
            {"role": "user", "content": "Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening"},
            {"role": "assistant", "content": "{'service':'positive','food':'positive','price':'positive'}"},
            {"role": "user", "content": text}
            ]
            )

    openai_response.append(response.choices[0].message.content)






In [ ]:
df["openai_no_finetune"] = openai_response

In [ ]:
df.to_csv("../../eval/results/openai/openai_result.csv")

## Finetuning

In [10]:
client.files.create(
  file=open("openai.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-yonRGbb8AexonWqklxVAm0g9', bytes=66837, created_at=1717325776, filename='openai.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [12]:
client.fine_tuning.jobs.create(
  training_file="file-yonRGbb8AexonWqklxVAm0g9", 
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-7EAIs5HSTSHnS69VMF9FwMac', created_at=1717325809, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-xE3l5iHAYqkV1PJDFAwfkWoo', result_files=[], seed=765579872, status='validating_files', trained_tokens=None, training_file='file-yonRGbb8AexonWqklxVAm0g9', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [16]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-7EAIs5HSTSHnS69VMF9FwMac', created_at=1717325809, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:infomedia::9VdLBKV7', finished_at=1717326420, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-xE3l5iHAYqkV1PJDFAwfkWoo', result_files=['file-qUVLZ1oPQT6uE1W4UfiKJSo6'], seed=765579872, status='succeeded', trained_tokens=35178, training_file='file-yonRGbb8AexonWqklxVAm0g9', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None), FineTuningJob(id='ftjob-w26DyboMYSqYt1eCwuKlvMM7', created_at=1717323794, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=5, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id=

In [18]:
# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-7EAIs5HSTSHnS69VMF9FwMac", limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-S1Eh9iChq338g6suXny9ICP3', created_at=1717326424, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-FZF9zRHgHHE1YGTOnrQ7WAHp', created_at=1717326422, level='info', message='New fine-tuned model created: ft:gpt-3.5-turbo-0125:infomedia::9VdLBKV7', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-IuXsutNHyiIjIbGd6IrZ1Jh5', created_at=1717326422, level='info', message='Checkpoint created at step 200 with Snapshot ID: ft:gpt-3.5-turbo-0125:infomedia::9VdLBwLk:ckpt-step-200', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-qUcwMylpNQk0x7wVzqy4R5OO', created_at=1717326422, level='info', message='Checkpoint created at step 100 with Snapshot ID: ft:gpt-3.5-turbo-0125:infomedia::9VdLBUwg:ckpt-step-100', object='fine_tuning.job.event', data={}, typ

In [19]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-7EAIs5HSTSHnS69VMF9FwMac")

FineTuningJob(id='ftjob-7EAIs5HSTSHnS69VMF9FwMac', created_at=1717325809, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:infomedia::9VdLBKV7', finished_at=1717326420, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-xE3l5iHAYqkV1PJDFAwfkWoo', result_files=['file-qUVLZ1oPQT6uE1W4UfiKJSo6'], seed=765579872, status='succeeded', trained_tokens=35178, training_file='file-yonRGbb8AexonWqklxVAm0g9', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

## Finetuned model

In [24]:
df = pd.read_csv("../../eval/results/openai/openai_result.csv", index_col=0)

In [25]:
df.columns

Index(['text', 'true_labels', 'openai_no_finetune'], dtype='object')

In [26]:
openai_response_ft = []

for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    label = row["true_labels"]
    text = row["text"]

    response = client.chat.completions.create(
        model="ft:MODEL_ID", 
        response_format={ "type": "json_object" }, 
        messages=[
            {"role": "system", "content": f"Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. Output should only contain mentioned aspects and their respective sentiments as a json. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation."},
            {"role": "user", "content": "Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening"},
            {"role": "assistant", "content": "{'service':'positive','food':'positive','price':'positive'}"},
            {"role": "user", "content": text}
            ]
            )

    openai_response_ft.append(response.choices[0].message.content)


  0%|          | 0/749 [00:00<?, ?it/s]

In [27]:
df["openai_finetuned"] = openai_response_ft

In [28]:
df.to_csv("../../eval/results/openai/openai_result.csv")